### Books Recommender System using clustering | Collaborative based

This Project show the recommendation of Books as per the user intrest

**Importing Libraries**

- `pandas` for data handling and manupulation.
- `numpy` for mathmatical calculations.
- `warnings` for ignoring the warnings during the progress.

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

_**Loading** the dataset with `books` variable_

In [2]:
books = pd.read_csv(r'data/BX-Books.csv',sep=';',on_bad_lines='warn',encoding='latin-1')

Skipping line 6452: expected 8 fields, saw 9
Skipping line 43667: expected 8 fields, saw 10
Skipping line 51751: expected 8 fields, saw 9

Skipping line 92038: expected 8 fields, saw 9
Skipping line 104319: expected 8 fields, saw 9
Skipping line 121768: expected 8 fields, saw 9

Skipping line 144058: expected 8 fields, saw 9
Skipping line 150789: expected 8 fields, saw 9
Skipping line 157128: expected 8 fields, saw 9
Skipping line 180189: expected 8 fields, saw 9
Skipping line 185738: expected 8 fields, saw 9

Skipping line 209388: expected 8 fields, saw 9
Skipping line 220626: expected 8 fields, saw 9
Skipping line 227933: expected 8 fields, saw 11
Skipping line 228957: expected 8 fields, saw 10
Skipping line 245933: expected 8 fields, saw 9
Skipping line 251296: expected 8 fields, saw 9
Skipping line 259941: expected 8 fields, saw 9
Skipping line 261529: expected 8 fields, saw 9



_While I normally loading this data I got unicode error to rectify that I used `encoding='latin-1'` Then 
after i running again i got pharse error in order to rectify that I used `on_bad_lines='warn'`_

Checking the data with `head` function

In [3]:
books.head(3)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...


I have observed the dataset and examined its features.
<br>which include:
<br>**ISBN (International Standard Book Number):** _The ISBN is a unique identifier for a book, facilitating the identification of a specific edition of a book._
<br>**Book-Title:** _This column represents the title of the book. It is the name given to the literary work, providing readers with a quick insight into the content of the book._
<br>**Book-Author:** _The Book-Author column contains the name of the author who wrote the book._
<br>**Year-Of-Publication:** _This column denotes the year when the book was originally published._
<br>**Publisher:** _The Publisher column specifies the publishing company responsible for producing and distributing the book._
<br>**mage-URL-S, Image-URL-M, Image-URL-L:** _These columns contain URLs pointing to different sizes of images associated with the book. The "S" refers to a small-sized image,<br> &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp;  "M" to a medium-sized image, and "L" to a large-sized image._

Now Checking for the shape of the dataset

In [4]:
books.shape

(271360, 8)

Printing the columns of dataset

In [5]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

_Taking a seperate DataFrame for required columns_

In [6]:
books = books[['ISBN','Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher','Image-URL-L']]

In [7]:
books.head(2)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...


_Renaming the columns for easy access_

In [8]:
books.rename(columns={
    'Book-Title':'title',
    'Book-Author':'author',
    'Year-Of-Publication':'year',
    'Publisher': 'publisher',
    'Image-URL-L':'img_url'},inplace=True)

In [9]:
books.head(2)

,ISBN,title,author,year,publisher,img_url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...


_Then **Loading** the data set of `Users` for checking any information is useful in that data set._

In [10]:
users = pd.read_csv(r'data/BX-Users.csv',sep=';',on_bad_lines='warn',encoding='latin-1')

In [11]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [12]:
users.shape

(278858, 3)

_Then **Loading** another data set of `Book Ratings`, This also checking for any information is useful._

In [13]:
ratings = pd.read_csv(r'data/BX-Book-Ratings.csv',sep=';',on_bad_lines='warn',encoding='latin-1')

In [14]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


_Checking the shape of the data set_

In [15]:
ratings.shape

(1149780, 3)

_Now printing the shape of each Data Set which I have loaded._

In [16]:
print(books.shape)
print(users.shape)
print(ratings.shape)

(271360, 6)
(278858, 3)
(1149780, 3)


_Observed that `books` and `users` have almost similar rows but `ratings` data set have more rows than other_
_<br>I want to check into that ratings data_

## Data Cleaning and Processing

_Renaming the Columns for easy access_

In [17]:
ratings.rename(columns={
    'User-ID':'user_id',
    'Book-Rating':'rating'},inplace=True)

In [18]:
ratings.head()

,user_id,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


Now printing the value counts of `user_id`

In [19]:
ratings['user_id'].value_counts()

user_id
11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: count, Length: 105283, dtype: int64

_Observed that user_id 11676 had given ratings to 13,602 books, while user_id 276723 had given a rating to just one book._

_I want to filter who was rated more than 200 books_

In [20]:
ratings['user_id'].unique().shape

(105283,)

_Now, considering x as the users who have rated more than 200 books._

In [21]:
x = ratings['user_id'].value_counts() > 200

In [22]:
x

user_id
11676      True
198711     True
153662     True
98391      True
35859      True
          ...  
116180    False
116166    False
116154    False
116137    False
276723    False
Name: count, Length: 105283, dtype: bool

In [23]:
x[x].shape

(899,)

_Considering y for only true values_

In [24]:
y = x[x].index
y

Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352, 110973,
       235105,
       ...
       260183,  73681,  44296, 155916,   9856, 274808,  28634,  59727, 268622,
       188951],
      dtype='int64', name='user_id', length=899)

_Now, redefining the ratings dataframe to include only users who have rated more than 200 books._

In [25]:
ratings = ratings[ratings['user_id'].isin(y)]

In [26]:
ratings.head()

,user_id,ISBN,rating
1456,277427,002542730X,10
1457,277427,0026217457,0
1458,277427,003008685X,8
1459,277427,0030615321,0
1460,277427,0060002050,0


In [27]:
ratings.shape

(526356, 3)

_Then merging the datasets `books` and `ratings` based on the 'ISBN' column._

In [28]:
ratings_with_books = ratings.merge(books, on='ISBN')

In [29]:
ratings_with_books.head()

,user_id,ISBN,rating,title,author,year,publisher,img_url
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...


_Checking the shape of merged dataset._

In [30]:
ratings_with_books.shape

(487671, 8)

_Taking a DataFrame `ratings_with_books` and performing a groupby operation on the `title` column. Then grouping the data based on the unique values in the 'title' column for only `rating` column._

In [36]:
num_rating = ratings_with_books.groupby('title')['rating'].count().reset_index()

In [37]:
num_rating.head()

,title,rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [38]:
num_rating['rating'].unique()

array([  2,   1,   3,   5,   4,   7,  12,   8,  15,  10,  35,  13,  21,
         9,   6,  38,  11,  76, 162,  47,  40,  25,  41, 124,  22,  29,
        32,  71,  24,  20,  54,  43,  19,  46, 115,  17,  77,  14,  23,
        16,  85,  61,  50,  42,  18,  53,  34,  28,  63,  30,  27,  55,
        67, 140,  49,  33,  65, 119, 228,  57,  31, 137,  39,  37,  78,
       210,  69,  97,  60,  51,  81,  56,  64,  48, 103,  83, 127,  26,
        84, 134,  94,  92,  68, 193,  58,  70,  62,  90, 105,  95,  86,
        66, 132, 118, 111,  79, 108,  74,  52,  44, 123,  36, 104, 277,
        73,  93,  88, 121,  45, 110, 120, 122,  59,  91, 109, 100,  72,
       161,  80,  87,  89,  99,  98, 131,  82, 135, 174, 183, 112, 125,
       138, 101,  96, 177, 175, 116, 185, 136,  75, 145, 146, 212, 143,
       130, 113, 149, 151, 128, 206, 117, 188, 142, 169, 179, 224, 107,
       227, 200, 213, 129, 181, 270, 230, 241, 159, 236, 163, 171, 133,
       148, 180, 168, 209, 186, 182, 126, 150, 114, 363], dtype=

Renaming the column of `num_rating` data

In [40]:
num_rating.rename(columns={
    'rating':'num_of_ratings'},inplace=True)

In [41]:
num_rating.head()

,title,num_of_ratings
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


_Merging the `ratings_with_books` DataFrame with the `num_rating` DataFrame based on the 'title' column._

In [43]:
final_rating = ratings_with_books.merge(num_rating,on='title')

In [44]:
final_rating.head(3)

,user_id,ISBN,rating,title,author,year,publisher,img_url,num_of_ratings
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82


In [45]:
final_rating.shape

(487671, 9)

_Filtering the dataset to include only books that have received more than 50 ratings as `final_rating`._

In [46]:
final_rating = final_rating[final_rating['num_of_ratings']>=50]

In [47]:
final_rating.head(3)

,user_id,ISBN,rating,title,author,year,publisher,img_url,num_of_ratings
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82


Now printing the sample of final_rating

In [49]:
final_rating.sample(7)

,user_id,ISBN,rating,title,author,year,publisher,img_url,num_of_ratings
103179,209756,0312971125,5,Crazy for You,Jennifer Crusie,2000,St. Martin's Press,http://images.amazon.com/images/P/0312971125.0...,68
64406,85757,0449005615,0,Seabiscuit: An American Legend,LAURA HILLENBRAND,2002,Ballantine Books,http://images.amazon.com/images/P/0449005615.0...,69
79614,174216,0060915544,9,The Bean Trees,Barbara Kingsolver,1989,Perennial,http://images.amazon.com/images/P/0060915544.0...,132
119793,187145,0380778556,0,Rebecca,Daphne Du Maurier,1994,Avon,http://images.amazon.com/images/P/0380778556.0...,68
29631,6575,0380002930,4,Watership Down,Richard Adams,1976,Avon,http://images.amazon.com/images/P/0380002930.0...,115
99891,16795,0385721420,0,Three Junes,JULIA GLASS,2003,Anchor,http://images.amazon.com/images/P/0385721420.0...,84
93531,157273,0671894455,0,FORREST GUMP (Movie Tie in),Winston Groom,1994,Pocket,http://images.amazon.com/images/P/0671894455.0...,70


In [50]:
final_rating.shape

(61853, 9)

_Verifying whether there are any duplicated values in the `final_rating` DataFrame based on columns ("title","user_id")._

In [52]:
final_rating[final_rating.duplicated(subset=['title','user_id'])]

,user_id,ISBN,rating,title,author,year,publisher,img_url,num_of_ratings
702,11676,0440977096,10,The Secret Garden,Frances Hodgson Burnett,1989,Laure Leaf,http://images.amazon.com/images/P/0440977096.0...,79
710,11676,0879236493,9,The Secret Garden,Frances Hodgson Burnett,1987,David R. Godine Publisher,http://images.amazon.com/images/P/0879236493.0...,79
717,35050,0439099390,0,The Secret Garden,Frances Hodgson Burnett,1999,Scholastic,http://images.amazon.com/images/P/0439099390.0...,79
721,174791,0439099390,0,The Secret Garden,Frances Hodgson Burnett,1999,Scholastic,http://images.amazon.com/images/P/0439099390.0...,79
722,230522,0439099390,0,The Secret Garden,Frances Hodgson Burnett,1999,Scholastic,http://images.amazon.com/images/P/0439099390.0...,79
...,...,...,...,...,...,...,...,...,...
223478,172030,0821744941,9,Dark Angel,Anna Grant,1994,Zebra Books,http://images.amazon.com/images/P/0821744941.0...,54
223483,153662,0505524147,0,Dark Angel,Cassandra Collins,2000,Love Spell,http://images.amazon.com/images/P/0505524147.0...,54
228875,113270,0743439651,0,Still Waters,Jennifer Lauck,2001,Atria,http://images.amazon.com/images/P/0743439651.0...,83
228876,162639,0743439651,0,Still Waters,Jennifer Lauck,2001,Atria,http://images.amazon.com/images/P/0743439651.0...,83


_Seen that there are 2003 duplicate rows in the DataFrame_
<br>_Now Dropping the duplicate columns_

In [53]:
final_rating.drop_duplicates(['title','user_id'],inplace=True)

In [54]:
final_rating.shape

(59850, 9)

_Creating a `pivot_table` named `book_pivot` from the `final_rating` DataFrame using the `user_id` column as the index, where the `titles` are set as the index and the `ratings` as the values._

In [55]:
book_pivot = final_rating.pivot_table(columns='user_id', index='title', values='rating')

In [56]:
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84 Charing Cross Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,7.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
You Belong To Me,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


_Checking for the Shape of the book_pivot_

In [57]:
book_pivot.shape

(742, 888)

_Now, filling the NaN values with 0, as these 0 values represent that the particular user has not rated that particular book._

In [55]:
book_pivot.fillna(0,inplace=True)

In [56]:
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**Compressed Sparse Row (CSR) Format:** _It is the way to represent sparse matrices. In CSR format, you store only the non-zero values along with their row and column indices._

In [57]:
from scipy.sparse import csr_matrix

_Importing the `csr_matrix` class from scipy.sparse because it is useful for efficiently handling matrices with a large number of zeros, as is the case in the `book_pivot` matrix._

In [62]:
book_sparse = csr_matrix(book_pivot)

In [63]:
book_sparse

<742x888 sparse matrix of type '<class 'numpy.float64'>'
	with 613988 stored elements in Compressed Sparse Row format>

In [60]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm='brute')

In [61]:
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [62]:
distance, suggestion = model.kneighbors(book_pivot.iloc[237,:].values.reshape(1,-1),n_neighbors=6)

In [63]:
distance

array([[ 0.        , 68.78953409, 69.5413546 , 72.64296249, 76.83098333,
        77.28518616]])

In [64]:
suggestion

array([[237, 240, 238, 241, 184, 536]], dtype=int64)

In [65]:
for i in range(len(suggestion)):
    print(book_pivot.index[suggestion[i]])

Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Sorcerer's Stone (Book 1)', 'Exclusive',
       'The Cradle Will Fall'],
      dtype='object', name='title')


In [66]:
book_pivot.index[237]

'Harry Potter and the Chamber of Secrets (Book 2)'

In [67]:
books_name = book_pivot.index

In [68]:
books_name

Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       '84 Charing Cross Road', 'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Civil Action', 'A Cry In The Night',
       ...
       'Winter Solstice', 'Wish You Well', 'Without Remorse',
       'Wizard and Glass (The Dark Tower, Book 4)', 'Wuthering Heights',
       'Year of Wonders', 'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"'],
      dtype='object', name='title', length=742)

In [69]:
# import pickle 
# pickle.dump(model,open('artifacts/model.pkl','wb'))
# pickle.dump(books_name,open('artifacts/books_name.pkl','wb'))
# pickle.dump(final_rating,open('artifacts/final_rating.pkl','wb'))
# pickle.dump(book_pivot,open('artifacts/fbook_pivot.pkl','wb'))

In [70]:
def recommend_book(book_name):
    book_id = np.where(book_pivot.index == book_name)[0][0]
    distance, suggestion = model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1),n_neighbors=6)
    for i in range(len(suggestion)):
        books = book_pivot.index[suggestion[i]]
        for j in books:
            print(j)

In [71]:
book_name = 'Harry Potter and the Chamber of Secrets (Book 2)'
recommend_book(book_name)

Harry Potter and the Chamber of Secrets (Book 2)
Harry Potter and the Prisoner of Azkaban (Book 3)
Harry Potter and the Goblet of Fire (Book 4)
Harry Potter and the Sorcerer's Stone (Book 1)
Exclusive
The Cradle Will Fall
